In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pickle
from sklearn.model_selection import train_test_split,GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
onehot_geo=OneHotEncoder()
geo_one_hot=onehot_geo.fit_transform(data[['Geography']]).toarray()
geo_one_hot=pd.DataFrame(geo_one_hot,columns=onehot_geo.get_feature_names_out(['Geography']))
data=pd.concat([data.drop('Geography',axis=1),geo_one_hot],axis=1)

X=data.drop('Exited',axis=1)
y=data['Exited']

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)   
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)
with open('scalar.pkl','wb') as file:
    pickle.dump(scaler,file)
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_geo,file)

In [12]:
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train_scaled.shape[1],)))
    for i in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [13]:
model=KerasClassifier(build_fn=create_model,neurons=32,layers=1,epochs=100,verbose=0)

In [14]:
param_grid={
    'neurons':[8,16,32,64],
    'layers':[1,2,3,4],
    'epochs':[50,100]
}

In [15]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=3,n_jobs=-1)
grid_result=grid.fit(X_train_scaled,y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

c:\Users\shard\OneDrive\Desktop\ANN_platform\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\shard\OneDrive\Desktop\ANN_platform\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: 0.8586245736537247 using {'epochs': 100, 'layers': 3, 'neurons': 8}
